# Pipeline ETL: Silver-to-Gold (DW)

Este notebook executa o processo de ETL (Extração, Transformação e Carga) para mover dados da camada Silver (uber_silver) para a camada Gold (Data Warehouse).

Lógica:
1.  Dimensões: Carga incremental (apenas insere novos registos).
2.  Factos: Carga completa (TRUNCATE + INSERT ... SELECT) otimizada em SQL.

## 0. Importação das Bibliotecas

Importa as bibliotecas necessárias para o pipeline.

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import logging
import os
import sys

## 1. Configuração do Logging

Configura o sistema de logging para registar informações, erros e o progresso do script em vez de usar print().

In [2]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    handlers=[
                        logging.StreamHandler(sys.stdout) 
                    ])

## 2. Ligação à Base de Dados

Define a função connect_to_postgres que estabelece a ligação à base de dados PostgreSQL. Esta usa variáveis de ambiente para as credenciais.

In [3]:
def connect_to_postgres():
    try:
        db_user = os.getenv('POSTGRES_USER', 'admin')
        db_password = os.getenv('POSTGRES_PASSWORD', 'admin')
        db_name = os.getenv('POSTGRES_DB', 'postgres')
        db_host = 'localhost'
        
        conn_string = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}"
        
        logging.info("Criando engine do Postgres...")
        engine = create_engine(conn_string)
        
        with engine.connect() as conn:
            logging.info("Ligação com o Postgres estabelecida com sucesso!")
        
        return engine
    
    except Exception as e:
        logging.error(f"Falha ao ligar/criar engine: {e}")
        return None

## 3. Configuração e Início do Pipeline

Define o esquema de destino (dw) e inicializa a ligação à base de dados.

In [4]:
SCHEMA_NAME = 'dw'
logging.info(f"A iniciar carga incremental para o esquema '{SCHEMA_NAME}'...")

engine = connect_to_postgres()

if engine is None:
    logging.critical("Ligação à base de dados falhou. A abortar o pipeline.")
    raise Exception("Falha na ligação à Base de Dados")

2025-11-07 22:00:30 - INFO - A iniciar carga incremental para o esquema 'dw'...
2025-11-07 22:00:30 - INFO - Criando engine do Postgres...
2025-11-07 22:00:30 - INFO - Ligação com o Postgres estabelecida com sucesso!


## 4. ETAPA 1: Atualização Incremental das Dimensões

Nesta etapa, atualizamos as tabelas de dimensão (dim_cus, dim_veh, dim_pay, dim_loc).

A lógica é incremental: 
1.  Lemos as chaves de negócio (ex: customer_id) já existentes no DW.
2.  Lemos as chaves de negócio da tabela uber_silver.
3.  Identificamos quais chaves existem na silver mas não no DW.
4.  Inserimos (append) apenas as chaves novas. A base de dados (via SERIAL) trata da geração da srk_.

### 4.1. A atualizar dw.dim_cus

In [13]:
try:
    logging.info("Atualizando 'dim_cus'...")
    dw_cus = pd.read_sql(f"SELECT DISTINCT customer_id FROM {SCHEMA_NAME}.dim_cus", engine)
    silver_cus = pd.read_sql("SELECT DISTINCT customer_id FROM dw.uber_silver WHERE customer_id IS NOT NULL", engine)
    
    new_cus = silver_cus[~silver_cus['customer_id'].isin(dw_cus['customer_id'])]
    
    if not new_cus.empty:
        new_cus.to_sql('dim_cus', engine, schema=SCHEMA_NAME, if_exists='append', index=False)
        logging.info(f"-> Inseridos {len(new_cus)} novos registos em 'dim_cus'.")
    else:
        logging.info("-> 'dim_cus' já estava atualizada.")
except Exception as e:
    logging.error(f"ERRO ao atualizar 'dim_cus': {e}")

2025-11-07 22:02:10 - INFO - Atualizando 'dim_cus'...
2025-11-07 22:02:14 - INFO - -> Inseridos 147580 novos registos em 'dim_cus'.


### 4.2. A atualizar dw.dim_veh

In [14]:
try:
    logging.info("Atualizando 'dim_veh'...")
    dw_veh = pd.read_sql(f"SELECT DISTINCT vehicle_type::TEXT FROM {SCHEMA_NAME}.dim_veh", engine)
    silver_veh = pd.read_sql("SELECT DISTINCT vehicle_type::TEXT FROM dw.uber_silver WHERE vehicle_type IS NOT NULL", engine)
    
    new_veh = silver_veh[~silver_veh['vehicle_type'].isin(dw_veh['vehicle_type'])]
    
    if not new_veh.empty:
        new_veh.to_sql('dim_veh', engine, schema=SCHEMA_NAME, if_exists='append', index=False)
        logging.info(f"-> Inseridos {len(new_veh)} novos registos em 'dim_veh'.")
    else:
        logging.info("-> 'dim_veh' já estava atualizada.")
except Exception as e:
    logging.error(f"ERRO ao atualizar 'dim_veh': {e}")

2025-11-07 22:02:16 - INFO - Atualizando 'dim_veh'...
2025-11-07 22:02:16 - INFO - -> Inseridos 7 novos registos em 'dim_veh'.


### 4.3. A atualizar dw.dim_pay

In [15]:
try:
    logging.info("Atualizando 'dim_pay'...")
    dw_pay = pd.read_sql(f"SELECT DISTINCT payment_method::TEXT FROM {SCHEMA_NAME}.dim_pay", engine)
    silver_pay = pd.read_sql("SELECT DISTINCT payment_method::TEXT FROM dw.uber_silver WHERE payment_method IS NOT NULL", engine)

    new_pay = silver_pay[~silver_pay['payment_method'].isin(dw_pay['payment_method'])]
    
    if not new_pay.empty:
        new_pay.to_sql('dim_pay', engine, schema=SCHEMA_NAME, if_exists='append', index=False)
        logging.info(f"-> Inseridos {len(new_pay)} novos registos em 'dim_pay'.")
    else:
        logging.info("-> 'dim_pay' já estava atualizada.")
except Exception as e:
    logging.error(f"ERRO ao atualizar 'dim_pay': {e}")

2025-11-07 22:02:20 - INFO - Atualizando 'dim_pay'...
2025-11-07 22:02:20 - INFO - -> Inseridos 5 novos registos em 'dim_pay'.


### 4.4. A atualizar dw.dim_loc

Esta dimensão usa uma chave composta (pickup_location, drop_location), pelo que a lógica usa um merge do Pandas para identificar os novos pares.

In [16]:
try:
    logging.info("Atualizando 'dim_loc'...")
    dw_loc = pd.read_sql(f"SELECT pickup_location, drop_location FROM {SCHEMA_NAME}.dim_loc", engine)
    silver_loc = pd.read_sql(
        "SELECT DISTINCT pickup_location, drop_location FROM dw.uber_silver "
        "WHERE pickup_location IS NOT NULL AND drop_location IS NOT NULL", 
        engine
    )
    
    new_loc = silver_loc.merge(
        dw_loc, 
        on=['pickup_location', 'drop_location'], 
        how='left', 
        indicator=True
    )
    new_loc = new_loc[new_loc['_merge'] == 'left_only'].drop(columns=['_merge'])
    
    if not new_loc.empty:
        new_loc.to_sql('dim_loc', engine, schema=SCHEMA_NAME, if_exists='append', index=False)
        logging.info(f"-> Inseridos {len(new_loc)} novos registos em 'dim_loc'.")
    else:
        logging.info("-> 'dim_loc' já estava atualizada.")
except Exception as e:
    logging.error(f"ERRO ao atualizar 'dim_loc': {e}")

2025-11-07 22:02:23 - INFO - Atualizando 'dim_loc'...
2025-11-07 22:02:24 - INFO - -> Inseridos 30556 novos registos em 'dim_loc'.


### 4.5. A atualizar dw.dim_time

Esta etapa garante que a dimensão de data esteja populada.

A lógica é idempotente:
1.  Insere a linha padrão 'N/A' se ela não existir.
2.  Encontra o intervalo (MIN e MAX) de datas na `uber_silver`.
3.  Usa `generate_series` do PostgreSQL para criar todas as linhas de data nesse intervalo.
4.  Usa `ON CONFLICT (srk_date) DO NOTHING` para inserir apenas as datas que ainda não existem.

In [17]:
dim_date_load_sql = f"""
-- ETAPA A: Garantir que a linha 'N/A' (para datas nulas) existe.
INSERT INTO {SCHEMA_NAME}.dim_date (
    srk_date, full_date, day_name, day_of_week, day_of_month, day_of_year,
    month_name, month_number, quarter_number, year_number, is_weekend, is_holiday
) VALUES (
    -1, '1900-01-01', 'N/A', 0, 0, 0, 'N/A', 0, 0, 0, FALSE, FALSE
)
-- Se a srk_date -1 já existir, não faz nada.
ON CONFLICT (srk_date) DO NOTHING;

-- ETAPA B: Popular o intervalo de datas com base nos dados reais
WITH date_range AS (
    -- Encontra o intervalo real de datas presentes na camada Silver
    SELECT 
        MIN(date_time::date) AS min_date,
        MAX(date_time::date) AS max_date
    FROM dw.uber_silver
    WHERE date_time IS NOT NULL
)
INSERT INTO {SCHEMA_NAME}.dim_date (
    srk_date, full_date, day_name, day_of_week, day_of_month, day_of_year,
    month_name, month_number, quarter_number, year_number, is_weekend, is_holiday
)
SELECT
    TO_CHAR(d, 'YYYYMMDD')::INTEGER AS srk_date,
    d AS full_date,
    
    TO_CHAR(d, 'FMDay') AS day_name,
    EXTRACT(ISODOW FROM d) AS day_of_week, 
    EXTRACT(DAY FROM d) AS day_of_month,
    EXTRACT(DOY FROM d) AS day_of_year,
    TO_CHAR(d, 'FMMonth') AS month_name,
    EXTRACT(MONTH FROM d) AS month_number,
    EXTRACT(QUARTER FROM d) AS quarter_number,
    EXTRACT(YEAR FROM d) AS year_number,
    EXTRACT(ISODOW FROM d) IN (6, 7) AS is_weekend,
    FALSE AS is_holiday
FROM
    generate_series(
        (SELECT min_date FROM date_range),
        (SELECT max_date FROM date_range),
        '1 day'::interval
    ) AS t(d)
-- Se a srk_date (ex: 20251107) já existir, não faz nada.
ON CONFLICT (srk_date) DO NOTHING;
"""

try:
    logging.info("A popular/atualizar 'dim_date' de forma idempotente...")
    with engine.begin() as conn:
        conn.execute(text(dim_date_load_sql))
    
    logging.info("-> 'dim_date' está populada e atualizada.")

except Exception as e:
    logging.critical(f"ERRO CRÍTICO ao popular 'dim_date': {e}")
    raise e

2025-11-07 22:02:28 - INFO - A popular/atualizar 'dim_date' de forma idempotente...
2025-11-07 22:02:28 - INFO - -> 'dim_date' está populada e atualizada.


## 5. ETAPA 2: Recarga da Tabela de Factos (fat_rid)

Esta é a etapa principal. Executa uma única consulta SQL que corre inteiramente no PostgreSQL.

1.  TRUNCATE: A tabela de factos é limpa.
2.  INSERT ... SELECT: A consulta SQL faz o JOIN entre a uber_silver e as dimensões (que acabámos de atualizar), buscando as srk_ corretas.
3.  COALESCE: Garante que, se um JOIN falhar, a srk_ seja preenchida com a chave padrão 'N/A' (semeadas no DW).

### 5.1. Definição da Query SQL da Tabela de Factos

In [ ]:
fact_load_sql = f"""
-- Etapa A: Limpar a tabela de factos
TRUNCATE TABLE {SCHEMA_NAME}.fat_rid RESTART IDENTITY;

-- Etapa B: Inserir na tabela de factos
INSERT INTO {SCHEMA_NAME}.fat_rid (
    -- Métricas e Factos
    date_time, booking_status, avg_vtat, avg_ctat, cancelled_by,
    reason_for_cancelling, incomplete_rides_reason, booking_value,
    ride_distance, driver_rating, customer_rating,
    
    -- Chaves Estrangeiras (SKs)
    srk_cus, srk_veh, srk_pay, srk_loc,
    srk_date -- A CHAVE DE DATA
)
WITH
-- Pré-calcula as chaves "N/A" 
default_keys AS (
    SELECT 
        (SELECT srk_cus FROM {SCHEMA_NAME}.dim_cus WHERE customer_id = 'N/A') AS srk_cus_na,
        (SELECT srk_veh FROM {SCHEMA_NAME}.dim_veh WHERE vehicle_type = 'N/A') AS srk_veh_na,
        (SELECT srk_pay FROM {SCHEMA_NAME}.dim_pay WHERE payment_method = 'N/A') AS srk_pay_na,
        (SELECT srk_loc FROM {SCHEMA_NAME}.dim_loc WHERE pickup_location = 'N/A') AS srk_loc_na
)
SELECT
    -- Campos diretos da Silver
    s.date_time, 
    (s.booking_status::TEXT)::dw.booking_status_enum,
    s.avg_vtat, 
    s.avg_ctat, 
    (s.cancelled_by::TEXT)::dw.cancelled_by_enum,
    (s.reason_for_cancelling::TEXT)::dw.cancellation_reason_enum,
    (s.incomplete_ride_reason::TEXT)::dw.incomplete_reason_enum,
    s.booking_value, 
    s.ride_distance, 
    s.driver_rating, 
    s.customer_rating,
    
    -- Chaves (SKs) buscadas das dimensões
    COALESCE(cus.srk_cus, def.srk_cus_na),
    COALESCE(veh.srk_veh, def.srk_veh_na),
    COALESCE(pay.srk_pay, def.srk_pay_na),
    COALESCE(loc.srk_loc, def.srk_loc_na),
    
    -- JOIN com a dim_date
    COALESCE(d.srk_date, -1) -- Usa -1 para datas nulas (semeado na dim_date)
FROM
    dw.uber_silver AS s
CROSS JOIN
    default_keys AS def -- Traz as chaves 'N/A' para a consulta
LEFT JOIN
    {SCHEMA_NAME}.dim_cus AS cus ON s.customer_id = cus.customer_id
LEFT JOIN
    {SCHEMA_NAME}.dim_veh AS veh ON s.vehicle_type::TEXT = veh.vehicle_type::TEXT
LEFT JOIN 
    {SCHEMA_NAME}.dim_pay AS pay ON s.payment_method::TEXT = pay.payment_method::TEXT
LEFT JOIN
    {SCHEMA_NAME}.dim_loc AS loc ON s.pickup_location = loc.pickup_location
                                AND s.drop_location = loc.drop_location
LEFT JOIN
    {SCHEMA_NAME}.dim_date AS d 
    ON TO_CHAR(s.date_time, 'YYYYMMDD')::INTEGER = d.srk_date;
"""


logging.info("Query SQL da 'fat_rid' definida.")

2025-11-07 22:03:41 - INFO - Query SQL da 'fat_rid' definida.


### 5.2. Execução da Carga da Tabela de Factos

Executa a query SQL de carga dentro de uma transação para garantir a atomicidade.

In [20]:
try:
    logging.info("Executando carga da 'fat_rid' (TRUNCATE + INSERT) ...")
    with engine.begin() as conn:
        conn.execute(text(fact_load_sql))
    
    logging.info("-> Carga da 'fat_rid' concluída com sucesso.")

except Exception as e:
    logging.critical(f"ERRO CRÍTICO ao recarregar 'fat_rid': {e}")
    logging.error("A tabela de factos pode estar vazia ou em estado inconsistente.")


2025-11-07 22:03:43 - INFO - Executando carga da 'fat_rid' (TRUNCATE + INSERT) ...
2025-11-07 22:03:53 - INFO - -> Carga da 'fat_rid' concluída com sucesso.


## 6. Limpeza e Conclusão

Fecha o pool de ligações do engine do SQLAlchemy.

In [21]:
if engine:
    engine.dispose()
    logging.info("Engine do SQLAlchemy fechado.")

logging.info("Processo ETL (Silver-to-Gold) concluído.")

2025-11-07 22:03:53 - INFO - Engine do SQLAlchemy fechado.
2025-11-07 22:03:53 - INFO - Processo ETL (Silver-to-Gold) concluído.
